# 01_EDA – Exploratory Data Analysis  
Demand Forecasting (Top‑Down & Bottom‑Up)

Este notebook sigue la fase *Data Understanding* de CRISP‑DM para el reto de predicción M+2.
Incluye:
1. **Carga y saneo** de los tres orígenes de datos.
2. **Análisis Top‑Down** (total negocio → familia → categoría).
3. **Análisis Bottom‑Up** (SKU → cliente, intermitencia, ABC/XYZ).
4. **Dataset integrado** (`df_all`) listo para modelado.


In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from ipywidgets import interact, widgets

# --- Paths ---
DATA_PATH = r"C:\Developer\Laboratorio_III\data"
SELL_IN_FILE = DATA_PATH + r"\sell-in.txt"
PRODUCT_FILE = DATA_PATH + r"\tb_productos.txt"
STOCK_FILE = DATA_PATH + r"\tb_stocks.txt"


In [2]:

def load_txt(path, **kwargs):
    return pd.read_csv(path, sep="\t", engine="python", **kwargs)

df_sales   = load_txt(SELL_IN_FILE)
df_prod    = load_txt(PRODUCT_FILE)
df_stock   = load_txt(STOCK_FILE)

# Cast 'periodo' to datetime if present
for df in [df_sales, df_stock]:
    if 'periodo' in df.columns:
        df['periodo'] = pd.to_datetime(df['periodo'], format='%b-%y', errors='coerce')

display(df_sales.head(), df_prod.head(), df_stock.head())


,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
0,NaT,10234,20524,0,2,0.05300,0.05300
1,NaT,10032,20524,0,1,0.13628,0.13628
2,NaT,10217,20524,0,1,0.03028,0.03028
3,NaT,10125,20524,0,1,0.02271,0.02271
4,NaT,10012,20524,0,11,1.54452,1.54452


,cat1,cat2,cat3,brand,sku_size,product_id
0,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20280
1,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20180
2,HC,ROPA LAVADO,Liquido,LIMPIEX,120,20332
3,HC,ROPA LAVADO,Liquido,LIMPIEX,450,20222
4,HC,ROPA LAVADO,Liquido,LIMPIEX,900,20288


,periodo,product_id,stock_final
0,NaT,20524,1.61267
1,NaT,20311,2.93657
2,NaT,20654,6.83269
3,NaT,21005,1.01338
4,NaT,20974,0.34595


In [ ]:

for name, df in [('Sales', df_sales), ('Productos', df_prod), ('Stock', df_stock)]:
    print(f"\n{name}:")
    display(df.info())
    display(df.isna().mean().rename('%NA'))


In [ ]:

df_all = (df_sales
          .merge(df_prod, on='product_id', how='left')
          .merge(df_stock, on=['product_id', 'periodo'], how='left')
         )

# Guardar para reutilizar
df_all.to_parquet(DATA_PATH + r"\dataset_integrado.parquet")
print("Dataset integrado shape:", df_all.shape)
df_all.head()


## Top‑Down Analysis  
Agregamos por **Total**, **cat1** y **periodo** para visualizar tendencias y estacionalidad.

In [ ]:

agg_family = (df_all
              .groupby(['periodo', 'cat1'], observed=True)['tn']
              .sum()
              .reset_index())

pivot = agg_family.pivot(index='periodo', columns='cat1', values='tn')
pivot.plot(figsize=(12,4))
plt.title('Toneladas vendidas por familia')
plt.ylabel('tn')
plt.show()

# Interactivo: seleccionar familia para zoom
@interact(familia=df_all['cat1'].dropna().unique())
def plot_familia(familia='HC'):
    ts = pivot[familia]
    ts.plot(figsize=(10,3))
    plt.title(f'Ventas {familia}')
    plt.show()


## Bottom‑Up Analysis  
Clasificación ABC/XYZ e intermitencia por SKU.

In [ ]:

# ABC based on cumulative contribution to sales
sku_tot = df_all.groupby('product_id')['tn'].sum().sort_values(ascending=False)
cum_pct = sku_tot.cumsum() / sku_tot.sum()
abc_class = pd.cut(cum_pct, bins=[0, .8, .95, 1], labels=list('ABC'))
abc_df = pd.DataFrame({'total_tn': sku_tot, 'cum_pct': cum_pct, 'ABC': abc_class})

# XYZ: CoV of monthly demand
cov = df_all.groupby('product_id')['tn'].apply(lambda x: x.std()/x.mean() if x.mean()!=0 else np.nan)
xyz_class = pd.cut(cov, bins=[-np.inf, .5, 1, np.inf], labels=list('XYZ'))
abc_df['XYZ'] = xyz_class

display(abc_df.head())

# Heatmap of ABC/XYZ matrix
matrix = pd.crosstab(abc_df['ABC'], abc_df['XYZ'])
sns.heatmap(matrix, annot=True, fmt='d')
plt.title('ABC‑XYZ matrix')
plt.show()


### Intermittency & Demand Patterns

In [ ]:

def calc_intermittency(series):
    demand_periods = (series > 0).sum()
    return 1 - demand_periods / len(series)

sku_inter = df_all.pivot_table(index='periodo', columns='product_id', values='tn', fill_value=0)
intermittency = sku_inter.apply(calc_intermittency).sort_values()

intermittency.hist(bins=20, figsize=(6,3))
plt.title('Distribución de intermitencia por SKU')
plt.xlabel('Intermittency (0‑1)')
plt.show()


## Guardado de outputs

In [ ]:

summary = {
    'total_skus': df_prod['product_id'].nunique(),
    'period_range': (df_all['periodo'].min(), df_all['periodo'].max()),
    'mean_intermittency': intermittency.mean()
}
print(summary)


## Próximos pasos  
- Engineering de features temporales (lags, rolling statistics).  
- Train/test split con ventanas deslizantes para horizonte +2.  
- Benchmarking de modelos básicos (Naïve, Seasonal Naïve, ETS).